# Enunciado

1)  Se  tiene  información  estadística  de  la  temporada 
regular  de  todos  los 
jugadores   de   la   NBA   en   un   RDD   de   tuplas   con   el   siguiente   formato: 
`(id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, 
promedio_bloqueos,  promedio_rebotes,  promedio_faltas)`.  Un  analista  de  la 
cadena ESPN  está  tra
bajando  con  un  RDD que corresponde  a  la primera  ronda 
de playoffs y que tiene el siguiente formato: `(id_jugador, id_partido, timestamp, 
cantidad_puntos, 
cantidad_rebotes, 
cantidad_bloqueos, 
cantidad_robos, 
cantidad_asistencias,   cantidad_faltas)`.   En   base   a 
estos   RDDs   se   quiere 
programar  en  PySpark  un  programa  que  **genere  un  RDD  con  los  nombres  (sin 
duplicados)  de  los  jugadores  que  lograron  en  algún  partido  de  playoffs  una 
cantidad de asistencias mayor a su promedio histórico**.  (\*\*\*\*) (15 pts)

---

# Pasamos rdd a formato (K,V)

No quedamos con `(id_jugador,(nombre,promedio_asistencias))` del rdd de promedios:

```
promedios_rdd = promedios_rdd.map(lambda x: (x[0],(x[1],x[3]))
```

Nos quedamos con `(id_jugador,cantidad_asistencias)` del rdd de playoffs:

```
playoffs_rdd = playoffs_rdd.map(lambda x: (x[0],x[7]))
```

# Hacemos el join

```
rdd = playoffs_rdd.left_join(promedios_rdd)
```

Hacemos un `left_join` ya que el rdd con más información es el de playoffs (más cantidad de registros).
Si lo hacemos para el otro lado, perdemos bastantes registros.

La estructura nos queda de la forma: `(id_jugador,(cantidad_asistencias,(nombre,promedio_asistencias)))`.

# Filtramos los datos

Nos quedamos con los que tienen más asistencias que su promedio:

```
rdd = rdd.filter(rdd[1][0] > rdd[1][1][1])
```

# Obtenemos los nombres

```
rdd = rdd.reduceByKey(lambda x,y: x)
         .map(lambda x: x[1][1][0])
         .collect()
```